### Load experiment metadata

In [4]:
from sumo_python.helpers.load_experiment_metadata import load_experiment_config

input_path = "/Users/rodrse/Documents/DemandCalibration/notebooks/od_calibration_sumo_files/"


In [5]:
import os
os.listdir(input_path)

['.DS_Store',
 'config.json',
 'simulation_setups.json',
 'quickstart',
 '.ipynb_checkpoints']

In [6]:
config, sim_setup = load_experiment_config(config=input_path + 'config.json',
                        sim_setup=input_path + 'simulation_setups.json',)

In [7]:

config

{'NETWORK': PosixPath('/Users/rodrse/Documents/DemandCalibration/notebooks/od_calibration_sumo_files/quickstart/data'),
 'SUMO': PosixPath('/usr/local/opt/sumo/share/sumo')}

In [8]:
sim_setup

{'objective': 'counts',
 'net': 'quickstart.net.xml',
 'route2od': 'quickstart.rou_for_od.xml',
 'taz': 'quickstart.taz.xml',
 'add': 'quickstart.additional.xml',
 'start_sim_sec': 54000,
 'end_sim_sec': 57600,
 'gt_od': 'quickstart.gt_od.xml',
 'current_od': 'quickstart.current_od.xml',
 'init_od': 'quickstart.init_od.xml',
 'prior_od': 'quickstart.prior_od.xml',
 'n_sumo_replicate': 1}

# Simulator

Steps:

1. Create OD meta-files that for a given count, creates a random trajectoy of vehicles moving from O -> D
2. Run SUMO against the latter files
3. Parse output of simulation and yields the aggregated value of vehicles which arrive to a given node in th network



### Step 1. Create OD metadata

In [9]:
import numpy as np
from sumo_python.helpers.generate_od_xml import generate_od_xml


prefix_run = 'iter'
#od_rand = np.random.rand(4)*200
od_rand = np.array([500,500,900,900])
#print(prefix_run, np.sum(od_rand))
    
generate_od_xml(od_rand, config, sim_setup, prefix_run)


printing /Users/rodrse/Documents/DemandCalibration/notebooks/od_calibration_sumo_files/quickstart/data/iter_quickstart.current_od.xml


### Step 2-3: Run SUMO and parse output

In [10]:
from sumo_python.simulator.run_sumo_and_parse_output import run_sumo_and_parse_output

df_simulated = run_sumo_and_parse_output(config, sim_setup, prefix_run)


### Runing: iter
od2trips --no-step-log  --spread.uniform --taz-files /Users/rodrse/Documents/DemandCalibration/notebooks/od_calibration_sumo_files/quickstart/data/quickstart.taz.xml --tazrelation-files /Users/rodrse/Documents/DemandCalibration/notebooks/od_calibration_sumo_files/quickstart/data/iter_quickstart.current_od.xml -o /Users/rodrse/Documents/DemandCalibration/notebooks/od_calibration_sumo_files/quickstart/data/iter_od_trips.trips.xml 
Success.
Step #57600.00 (6ms ~= 166.67*RT, ~109333.33UPS, vehicles TOT 2800 ACT 656 BUF 0)         


In [11]:
df_simulated

,EdgeID,interval_begin,interval_end,interval_nVehContrib,interval_harmonicMeanSpeed
0,#EdgeL5,54000.0,57600.0,959.0,13.500000
1,#EdgeL7,54000.0,57600.0,1243.0,13.603333


### Obtain ground-truth

In [12]:
from sumo_python.simulator.ground_truth import run_ground_truth_and_parse
df_gt_data = run_ground_truth_and_parse(config, sim_setup)

od2trips --no-step-log  --spread.uniform --taz-files /Users/rodrse/Documents/DemandCalibration/notebooks/od_calibration_sumo_files/quickstart/data/quickstart.taz.xml --tazrelation-files /Users/rodrse/Documents/DemandCalibration/notebooks/od_calibration_sumo_files/quickstart/data/quickstart.gt_od.xml -o /Users/rodrse/Documents/DemandCalibration/notebooks/od_calibration_sumo_files/quickstart/data/gt_od_trips.trips.xml 
Success.
sumo --output-prefix gt_ --ignore-route-errors=true --net-file=/Users/rodrse/Documents/DemandCalibration/notebooks/od_calibration_sumo_files/quickstart/data/quickstart.net.xml --routes=/Users/rodrse/Documents/DemandCalibration/notebooks/od_calibration_sumo_files/quickstart/data/gt_od_trips.trips.xml -b 54000 -e 57600 --additional-files /Users/rodrse/Documents/DemandCalibration/notebooks/od_calibration_sumo_files/quickstart/data/quickstart.additional.xml --duration-log.statistics --xml-validation never --vehroutes /Users/rodrse/Documents/DemandCalibration/notebooks

In [13]:
df_gt_data

,EdgeID,interval_begin,interval_end,interval_nVehContrib,interval_harmonicMeanSpeed
0,#EdgeL5,54000.0,57600.0,959,13.500000
1,#EdgeL7,54000.0,57600.0,1243,13.603333


## Compute loss

In [14]:
from sumo_python.helpers.nrmse_counts import compute_nrmse_counts

loss_estimate = compute_nrmse_counts(df_gt_data, df_simulated)
loss_estimate

0.0